In [1]:
%pylab inline
import pandas as pd
import numpy as np

import torch
from torchvision import transforms
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

import torch.optim as optim                          # optimization

import os
from tqdm import tqdm                                # for progress bar
from sklearn.model_selection import train_test_split
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import adjusted_rand_score

from clustering import clus

from numba import njit

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("running on the gpu")
else:
    device = torch.device("cpu")
    print("running on the cpu")
    
from compression import pipeline

Populating the interactive namespace from numpy and matplotlib
running on the gpu


In [6]:
path = "goolam"
do_clus = True
latent = pipeline(path, False, vae_choice='paper', retrain=True, seed=1) # a list with 3 differently trained latent variables
print(len(latent), latent[0].shape)


log2 is applied
training non-negative kernel autoencoder
epoch 9 Loss:  0.1918240785598755
epoch 9 Loss:  0.1924905776977539
epoch 9 Loss:  0.19195954501628876
tensor([0.0404, 0.1212, 0.0171,  ..., 0.1827, 0.2033, 0.0587])
paper_encoder(
  (encoder): Linear(in_features=124, out_features=64, bias=True)
  (bn): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  (mu): Linear(in_features=64, out_features=15, bias=True)
  (var): Linear(in_features=64, out_features=15, bias=True)
  (sample_expander): ModuleList(
    (0): Linear(in_features=15, out_features=64, bias=True)
    (1): Linear(in_features=15, out_features=64, bias=True)
  )
  (decoder): ModuleList(
    (0): Linear(in_features=64, out_features=124, bias=True)
    (1): Linear(in_features=64, out_features=124, bias=True)
  )
)
training stacked bayesian autoencoder

###############################
#phase 1: the warm-up process#
######################################
epoch 9 Loss: 0.14472079277038574

###

## classification

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial.distance import correlation
from scipy.stats import mode
import random

path_name = 'goolam'
y_df = pd.read_csv(f'rds_csv_data/{path_name}_labels.csv')['x']
y = np.array(y_df)

# 80% train, 20% test
spilt = int(len(y)*0.8)

print(spilt)
results = []
for x in latent:
    random.seed(1)
    x_train, y_train, x_test, y_test =x[:spilt], y[:spilt], x[spilt:], y[spilt:]
    knn = KNeighborsClassifier(n_neighbors=10, metric=correlation, n_jobs=-1)
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)
    results.append(y_pred)

result = mode(results)
print((result.mode==y_test).mean())
    
    

99
0.8
['2cell' '4cell' '4cell' '4cell' '4cell' '4cell' '4cell' '4cell' '4cell'
 '4cell' '4cell' '4cell' '4cell' '4cell' '4cell' '4cell' '4cell' '4cell'
 '4cell' '4cell' '4cell' '4cell' '4cell' '4cell' '4cell']


In [3]:

if do_clus:
    # Use an ensemble of data projection models to achieve higher accuracy and to avoid local minima, not needed if we use kmeans++
    # first repeat the data projection
    labels = []
    for hidden in latent:
        # labels.append(clus(hidden, k=6, nmax=100))
        labels.append(clus(hidden, nmax=50))
    labels = np.array(labels) 
    print(labels)   
    S = np.zeros((len(labels), len(labels)))  # chance that cell i and j are in the same cluster
    for i, row in enumerate(S):
        for j, _ in enumerate(S):
            if not (i==j):
                S[i, j] = adjusted_rand_score(labels[i], labels[j])
    for i, row in enumerate(S):
        S[i,i] = row.mean()
    print(S)
    found = False
    if (S[S < 0.7]).sum() > 0:
        i = 2
    else:
        i = 1
        
    # find best guessed label (latent variable)
    while not found:
        # print(f'i={i}')
        tmp = KMeans(n_clusters = i, n_init = 100, max_iter = 5000).fit(S)
        k = tmp.labels_
        max = 0
        for c in range(tmp.cluster_centers_.shape[0]): # for k clusters
            score = S[k == c, k == c].mean()
            if score > max and (k==c).sum() > 1:
                max = score
                idx = (k == c)
        if max > 0.8:
            found = True
        if i >= 3:
            found = True
        
        i += 1
    # guess number of clusters
    tmp = []
    for label in labels[idx]:
        tmp.append(np.unique(label).shape[0])
        print(tmp)
    cluster_max = np.floor(np.mean(tmp)+0.5).astype(np.int)
    
    
    # (i) calculate cell-cell weighted similarity matrix 
    W = S * (1 - S)
    print(W.max(), W.min())
    # then combine the clustering results using the wMetaC
    # wMetaC = AgglomerativeClustering(n_clusters=k_classes, linkage='ward')
    # # wMetaC = AgglomerativeClustering(n_clusters=k_classes, affinity='precomputed')
    # wMetaC.fit(latent)
    # # wMetaC.fit(clustered.affinity_matrix_.toarray())
    # print(wMetaC)
    # print(clustered.labels_)
    # print(wMetaC.labels_)

        # print(latent.size())
    
    


(1600, 15)
finding best k...
best k is 5
running SC on k=5...
SpectralClustering(affinity='nearest_neighbors', eigen_solver='arpack',
                   n_clusters=5, n_jobs=-1, n_neighbors=7, random_state=0)
(1600, 15)
finding best k...
best k is 5
running SC on k=5...
SpectralClustering(affinity='nearest_neighbors', eigen_solver='arpack',
                   n_clusters=5, n_jobs=-1, n_neighbors=7, random_state=0)
(1600, 15)
finding best k...
best k is 5
running SC on k=5...
SpectralClustering(affinity='nearest_neighbors', eigen_solver='arpack',
                   n_clusters=5, n_jobs=-1, n_neighbors=7, random_state=0)
[[1 3 3 ... 3 1 3]
 [0 0 0 ... 0 3 1]
 [2 0 2 ... 3 3 4]]
[[ 4.09367810e-05  1.27577776e-03 -1.15296742e-03]
 [ 1.27577776e-03  6.17736990e-04  5.77433208e-04]
 [-1.15296742e-03  5.77433208e-04 -1.91844737e-04]]


NameError: name 'idx' is not defined

In [4]:
pipeline('campbell', 1000, 3, False, 1000, 5, vae_choice='paper')

training non-negative kernel autoencoder


100%|██████████| 22/22 [00:00<00:00, 106.05it/s]


Loss:  0.001624945318326354


100%|██████████| 22/22 [00:00<00:00, 107.14it/s]


Loss:  0.0016046841628849506


...

100%|██████████| 22/22 [00:00<00:00, 139.61it/s]


Loss:  1.9809030294418335


100%|██████████| 22/22 [00:00<00:00, 146.08it/s]


Loss:  1.951000690460205


tensor([[-0.0781,  0.0789,  0.0101,  ...,  0.0791, -0.1060,  0.0511],
        [-0.0928,  0.0692,  0.0293,  ...,  0.0207, -0.0778,  0.0243],
        [-0.0822,  0.0621, -0.0014,  ...,  0.0103, -0.1345,  0.0899],
        ...,
        [-0.0850,  0.0382, -0.0398,  ...,  0.0492, -0.0836,  0.1068],
        [-0.0944,  0.0489, -0.0496,  ...,  0.0135, -0.0710,  0.1006],
        [ 0.0677,  0.8861,  0.1220,  ...,  0.1088, -0.0515, -0.2227]],
       device='cuda:0')